# Content and Objective

+ Show BER of an uncoded BPSK modulated signal over an AWGN channel
+ BER is calculated using a Monte-Carlo simulation and compared to theoretic performance
+ Confidence interval of the Monte-Carlo simulation is included based on statistical properties of the binomial distribution. 
+ Optimal decision threshold (based on MAP) derived for non-uniform distributed bits at the bottom of the notebook.

# Import

In [16]:
import numpy as np
from scipy import special
import matplotlib.pyplot as plt
import ipywidgets as widgets


# Some basic definitions: BPSK modulation, AWGN channel, Q-Funktion
+ SNR for AWGN: $\frac{E_s}{N_0}=\frac{E_s}{2\cdot \sigma_n^2}=\frac{1}{\sigma_n^2}$ for $E_s=1$
+  Q-Funktion: $Q(x)=\int_x^{\infty}\frac{1}{\sqrt{2\pi}}\exp\left(-\frac{y^2}{2}\right) dy$



# Functions

In [17]:
#BPSK mod
def bpsk_mod(x):
    return np.where(x == 0, 1, -1)

#AWGN channel
def awgn(x, sigma):
    noise = sigma*np.random.randn(len(x))
    return x+noise

#Q function
def qfunc(x):
    return 0.5-0.5*special.erf(x/np.sqrt(2))

#Optimal curve (See MAP derivation at the end of the notebook)
def threshold_calc(p_0, sigma):
    return np.log((1-p_0)/p_0)*(sigma**2)/2

vec_treshold_calc=np.vectorize(threshold_calc)

vec_qfunc=np.vectorize(qfunc)


# Choose here probability of 0s and 1s:

In [18]:
p_1 = 0.7  #Adapt in 0.01 steps: Probability for binary one --> After BPSK modulation it becomes a +1 
p_0 = 1-p_1


# Simulation + Plotting function

In [19]:
def plot_hist_bpsk(es_no_dB, N_syms_log, equally_likely_bits):
    
    #dB into linear
    es_no = 10**(es_no_dB/10)
    sigma = np.sqrt(1/(2*es_no))  

    #calculate decision treshold based on MAP
    current_threshold=threshold_calc(p_0,sigma=sigma)

    #Chosen number of simulations
    num_sim = int(10**(N_syms_log))

    # Generate random Data according to given probabilities
    binary_data = np.random.randint(low=0, high=1000, size=num_sim)
    binary_data = np.where(binary_data < 1000*p_1, 1, 0)

    
    

    ##Simulated Transmission (Monte-Carlo simulation)

    #BPSK modulation
    modulated = bpsk_mod(binary_data)

    #AWGN Channel
    transmitted = awgn(modulated, sigma)

    #For colouring seperate data with knowledge of what was transmitted
    transmitted_1 = transmitted[binary_data == 0]
    transmitted_minus_1 = transmitted[binary_data == 1]

    ## Theory:

    #Simulated SNR regime
    eb_no_range_db = np.linspace(-5, 10, 1000)
    eb_no_range = 10**(eb_no_range_db/10)

    # ML curve
    theoretic_ber_gv = qfunc(np.sqrt(2*eb_no_range))
    sigma_range = np.sqrt(1/(2*eb_no_range))

    decision_threshold = vec_treshold_calc(p_0, sigma_range) 

    #MAP
    theoretic_ber_non_gv = p_0 * \
        (1-vec_qfunc(((decision_threshold-1)/sigma_range))) + \
        p_1*vec_qfunc((decision_threshold+1)/sigma_range)

    
    #Plotting and Coloruing
    plt.figure(figsize=(20, 7.5))

    ax = plt.subplot(1, 4, 1)
    ax.set_title('Histogramm of the transmitted 1s and 0s')
    N_1, bins_1, patches_1 = plt.hist(transmitted_1,
                                      density=True, color='b', bins='auto',label='$f(x|1)$')  # np.arange(-6, 4+bin_width, bin_width
    N_minus_1, bins_minus_1, patches_minus_1 = plt.hist(transmitted_minus_1,
                                                        density=True, color='g', bins='auto', alpha=0.4,label='$f(x|-1)$')  # np.arange(-4, 6+bin_width, bin_width)
    
    plt.legend()
    plt.xlabel('x')
    ax = plt.subplot(1, 4, 2)
    plt.hist(transmitted, density=True, color='b', bins='auto',label='$f(x)$')
    ax.set_title('Histogramm of all received values combined')
    plt.legend()
    plt.xlabel('x')
    # print(bins_1)
    col_treshold = 'r'

    # Here trick to color area underneath histograms
    bins_1 = bins_1[1:]  
    bins_minus_1 = bins_minus_1[1:]  # deletes last element from bins

    # Monte Carlos --> Counter errors (Happens if threshold is crossed)
    errors = len(np.where(transmitted_1 <= current_threshold)[
                 0]) + len(np.where(transmitted_minus_1 >= current_threshold)[0])

    # BER= errors/numsim
    simulated_ber = float(errors)/num_sim
    
    ## HOW TO GET THE 2 sigma interval!
    # Simulated_ber corresponds to probabilty that error occurs 
    #--> can be used to calculate variance of a binomial distribution with simulated_ber being probability for one realization
    theoretic_var = simulated_ber*(1-simulated_ber)/num_sim

    # plot confidence area around estimated point
    theoretic_1_sigma_area = np.array(
        [simulated_ber+2*np.sqrt(theoretic_var), simulated_ber-2*np.sqrt(theoretic_var)])


    ## Now only plooting
    for i in range(len(np.where(bins_1 < current_threshold)[0])):
        patches_1[i].set_facecolor(col_treshold)

    for i in range(len(patches_minus_1)-1, len(patches_minus_1)-1-len(np.where(bins_minus_1 > current_threshold)[0]), -1):
        patches_minus_1[i].set_facecolor(col_treshold)

    ax = plt.subplot(1, 4, 3)
    ax.set_title('Complete theoretic curve')

    if(equally_likely_bits == "Theorie für Gleichverteilung"):
        plt.plot(eb_no_range_db, theoretic_ber_gv)
    elif(equally_likely_bits == "Theorie für nicht gleichverteilt"):
        plt.plot(eb_no_range_db, theoretic_ber_non_gv)
    plt.scatter(es_no_dB, simulated_ber, marker='x', color='r')
    plt.yscale('log')
    plt.xlabel('$E_b/N_0$')
    plt.ylabel('BER')
    plt.grid()

    ax = plt.subplot(1, 4, 4)
    ax.set_title('Zoom in on theoretic curve with 2 sigma area')

    if(equally_likely_bits == "Theorie für Gleichverteilung"):
        plt.plot(eb_no_range_db, theoretic_ber_gv)
    elif(equally_likely_bits == "Theorie für nicht gleichverteilt"):
        plt.plot(eb_no_range_db, theoretic_ber_non_gv)

    plt.scatter(es_no_dB, simulated_ber, marker='x', color='r')
    plt.plot(np.array([es_no_dB, es_no_dB]),
             theoretic_1_sigma_area, marker="_", color='g')

    # Ändern zu +-3 sigma umgebung
    plt.ylim(simulated_ber-3*np.sqrt(theoretic_var), simulated_ber+3*np.sqrt(theoretic_var))
    plt.yscale('log')
    plt.xlim(es_no_dB-1, es_no_dB+1)
    plt.ylabel('BER')
    plt.xlabel('$E_b/N_0$')
    plt.grid()


# Now the plotting...
+ SNR in dB can be adapted, as well as the number of simulated transmitted bits
+ First plot shows normalized histograms of the conditional pdfs of the received values given which bit was sent
  + The red colored areas are received values that were pushed beyond the decision threshold by the AWGN channel
+ Second plot shows the histogram of all received values
  + In case of non-uniform distributed bits the second plot illustrates that one transmitted value is more likely. Then, the adaptive threshold is beneficial.
+ Third plot shows the theoretic performance.
  + Simulated BER from Monte-Carlo simulation for current SNR added as red dot.
  + Theoretic curve can be chosen between MAP (=Theorie für nicht gleichverteilt) an ML (=Theorie für gleichverteilt).
+  In most communication system bits are uniform distributed and thus ML and MAP perform the same.

In [20]:
w = widgets.interact(plot_hist_bpsk,
                     es_no_dB=widgets.FloatSlider(min=-5, max=10),
                     N_syms_log=widgets.FloatSlider(min=2, max=7, value=5),
                     equally_likely_bits=widgets.RadioButtons(placeholder='Choose', options=["Theorie für Gleichverteilung", "Theorie für nicht gleichverteilt"]))


interactive(children=(FloatSlider(value=0.0, description='es_no_dB', max=10.0, min=-5.0), FloatSlider(value=5.…

## Derivation theoretic BER for non-uniform distributed bits: 
* BPSK modulation: $s_i=(-1)^{c_i}$
    + $P(s_i=1)=P(c_i=0)=:p_0$ und $P(s_i=-1)=P(c_i=1)=:p_1$
* AWGN channel: $z_i=s_i+n_i$ mit $n_i\sim \mathcal{N}(0,\sigma)$ 
* Decision threshold $\gamma_0$

\begin{align*}P(Fehler)&=P(s_i=1)\cdot P(z<\gamma_0|s_i=1) + P(s_i=-1)\cdot P(z>\gamma_0|s_i=-1)\\
&= p_0 \cdot (1-P(z\geq \gamma_0|s_i=1)) + p_1 \cdot (P(z>\gamma_0|s_i=1)\\
&= p_0 \cdot \left(1- \int_{\gamma_0}^{\infty}\frac{1}{\sqrt{2\pi\sigma^2}})\exp\left(-\frac{(z-1)^2}{2\sigma^2}\right) \right ) + p_1 \cdot \int_{\gamma_0}^{\infty}\frac{1}{\sqrt{2\pi\sigma^2}})\exp\left(-\frac{(z+1)^2}{2\sigma^2}\right)
\end{align*}

* Substituting $r=\frac{\gamma_0-1}{\sigma}$ and $y=\frac{\gamma_0+1}{\sigma}$ the integrals can be transformed, such that the definition of the Q-Function can be used:
\begin{align*}
P(Fehler)&= p_0\cdot \left(1-\int_{\frac{\gamma_0-1}{\sigma}}^\infty \frac{1}{\sqrt{2\pi}}\exp\left(-\frac{r^2}{2}\right) dr \right) + p_1\cdot \int_{\frac{\gamma_0+1}{\sigma}}^\infty \frac{1}{\sqrt{2\pi}}\exp\left(-\frac{y^2}{2}\right) dy \\
&= p_0 \cdot (1-Q\left(\frac{\gamma_0-1}{\sigma}\right)+ p_1 \cdot Q\left(\frac{\gamma_0+1}{\sigma}\right)
\end{align*}